In [1]:
import { WalletDecorator } from './WalletDecorator'
await WalletDecorator.waitReady()
import { tokens, tokensInfo } from './WalletDecorator'
const ethers = require('ethers')
import * as utils from './utils'

In [2]:
tokens

[ '0xef81a0bb8819f94eef15992f31555a282d7009d1', 'ETH' ]


# a breakthrough in manual testing using repl

# a lot of wallets send deposits

In [3]:
const numWallets = 16;

In [4]:
const wallets = await Promise.all(utils.rangearr(numWallets).map(WalletDecorator.fromId));

wallet sync:2708a49abc4d6464044abfb4b9bce3af69a2b55d syncNonce 0, ethNonce 232
wallet sync:6faa6414ebc8955a3abda739bb813794373ef0d8 syncNonce 0, ethNonce 17
wallet sync:b0b86e2187b6c6b75648c4a9e295f3da1b0426ab syncNonce 0, ethNonce 17
wallet sync:a8bba0ffab71a1fb9e7747e346bf19fa48f00a76 syncNonce 0, ethNonce 17
wallet sync:c59153f81862f7421aa11ca5726bac5c4d7b70d4 syncNonce 0, ethNonce 17
wallet sync:a75b6332b192c08652d458f80aa7e83b4fb40a7a syncNonce 0, ethNonce 17
wallet sync:f0e19f3dca0b4b10028c317d76a082a324264614 syncNonce 0, ethNonce 17
wallet sync:8acaec1019ae27a7edc6bed313e61d5b4c2b1853 syncNonce 0, ethNonce 17
wallet sync:51f0966bb81d77d0a67a9ece43f7b6c1839dc759 syncNonce 0, ethNonce 17
wallet sync:246c8892f369d4b8ce3cbccbfe983c88fb7f814d syncNonce 0, ethNonce 17
wallet sync:ca51243fbe3423b34ef5bff9d54554ac58f323cb syncNonce 0, ethNonce 17
wallet sync:e91c6b328fe227408ea142918a845caedef197a6 syncNonce 0, ethNonce 17
wallet sync:bd67e2867789e7a578d97679ba8152d1d64872fc syncNonce 

In [5]:
await wallets[0].prettyPrintBalances(tokens);
const amount = ethers.utils.parseEther('0.01');

Balance of 0x36615Cf349d7F6344891B1e7CA7C72883F5dc049 ( sync:2708a49abc4d6464044abfb4b9bce3af69a2b55d ):
┌────────────────────────────────────────────┬────────────────┬───────┬──────────┐
│                  (index)                   │      eth       │ sync  │ withdraw │
├────────────────────────────────────────────┼────────────────┼───────┼──────────┤
│ 0xef81a0bb8819f94eef15992f31555a282d7009d1 │ '3000000000.0' │ '0.0' │  '0.0'   │
│                    ETH                     │  '38.3949329'  │ '0.0' │  '0.0'   │
└────────────────────────────────────────────┴────────────────┴───────┴──────────┘


In [6]:
const txs = await wallets[0].mainchainSendToMany(wallets.slice(1), tokens, [amount.mul(2)]);

In [7]:
function depositMany(wallets, tokens, amounts) {
    const promises = [];
    for (const [wallet, token, amount] of utils.product(wallets, tokens, amounts)) {
        promises.push(wallet.deposit(amount, [token]));
    }
    return promises;
}

In [8]:
const promises = depositMany(wallets.slice(1), tokens.reverse(), [amount]);

In [9]:
const results = utils.flat(await Promise.all(promises));
const resultsOk = results.filter((tx: any) => !tx.error);

In [10]:
console.log('results.length', results.length, /* results.slice(-2) */);
console.log('resultsOk.length', resultsOk.length, /* resultsOk.slice(-2) */);

results.length 30
resultsOk.length 30


In [11]:
results.slice(5, 6)

[ { payload:
     { depositFrom: [Wallet],
       depositTo: [Wallet],
       token: '0xef81a0bb8819f94eef15992f31555a282d7009d1',
       amount: [BigNumber],
       overrideOptions: [Object] },
    tx:
     ETHOperation {
       ethTx: [Object],
       zkSyncProvider: [Provider],
       state: 'Committed',
       priorityOpId: [BigNumber] },
    receipt: { block: [Object], executed: true },
    error: null } ]


In [12]:
await WalletDecorator.revertReason('0x32b4a95eb0cf094ea0c3557f3a5c5b2dfbafb1b5a7f76c699a7776ddd1db7e84')

tx null


In [13]:
await wallets[0].prettyPrintBalances(tokens);

Balance of 0x36615Cf349d7F6344891B1e7CA7C72883F5dc049 ( sync:2708a49abc4d6464044abfb4b9bce3af69a2b55d ):
┌────────────────────────────────────────────┬────────────────┬───────┬──────────┐
│                  (index)                   │      eth       │ sync  │ withdraw │
├────────────────────────────────────────────┼────────────────┼───────┼──────────┤
│                    ETH                     │ '38.078993873' │ '0.0' │  '0.0'   │
│ 0xef81a0bb8819f94eef15992f31555a282d7009d1 │ '2999999999.7' │ '0.0' │  '0.0'   │
└────────────────────────────────────────────┴────────────────┴───────┴──────────┘


#### call this one before it says `pgs11`:

In [23]:
await wallets[0].cancelOutstandingDepositsForExodusMode(10, { gasLimit: 1000000 })
    .catch(x => WalletDecorator.revertReason(x.transaction.hash));

pgs11


In [24]:
await wallets[1].prettyPrintBalances(tokens);

Balance of 0xa61464658AfeAf65CccaaFD3a512b69A83B77618 ( sync:6faa6414ebc8955a3abda739bb813794373ef0d8 ):
┌────────────────────────────────────────────┬───────────────┬────────┬──────────┐
│                  (index)                   │      eth      │  sync  │ withdraw │
├────────────────────────────────────────────┼───────────────┼────────┼──────────┤
│                    ETH                     │ '0.020592834' │ '0.01' │  '0.01'  │
│ 0xef81a0bb8819f94eef15992f31555a282d7009d1 │    '0.01'     │ '0.01' │  '0.01'  │
└────────────────────────────────────────────┴───────────────┴────────┴──────────┘


In [25]:
function calculateDepositAmounts(results) {
    // compute total deposit amount for each token
    const sentAmounts = {};
    for (const { payload: { depositTo, token, amount } } of results) {
        const address = depositTo.address();
        sentAmounts[address] = (sentAmounts[address] || {});
        sentAmounts[address][token] 
            = (sentAmounts[address][token] || ethers.utils.bigNumberify(0));

        sentAmounts[address][token] = sentAmounts[address][token].add(amount);
    }
    for (const address of Object.keys(sentAmounts)) {
        for (const token of Object.keys(sentAmounts[address])) {
            sentAmounts[address][token] = ethers.utils.formatEther(sentAmounts[address][token]);
        }
    }
    return sentAmounts;
}


In [26]:
const sentAmounts = calculateDepositAmounts(resultsOk);

In [27]:
const addressToBalanceMap = Object.assign(
    {}, ...await Promise.all(
        wallets.map(
            async w => ({ [w.syncWallet.address()]: await w.balances(tokens) })
        )
    )
);

#### if this doesn't print anything, the test passes:

In [28]:
for (const address of Object.keys(sentAmounts)) {
    for (const token of Object.keys(sentAmounts[address])) {
        const real = await addressToBalanceMap[address][token]['withdraw'];
        const want = sentAmounts[address][token];
            if (real != want) {
            console.log('fuckup:', address, token, real, want);
        }
    }
}

# strange buggy
if the contract is deployed the first time (on fresh geth),  
sending 30 deposits from 15 wallets doesn't work

In [6]:
const cliProgress = require('cli-progress');

In [3]:
const NUM_WALLETS    = 64;
const DEPOSIT_AMOUNT = ethers.utils.parseEther('0.1');


In [4]:
await WalletDecorator.waitReady();

WalletDecorator.waitExodus('print'); // it'll notify us when it's exodus mode, no need to await

const wallet = await WalletDecorator.fromId(0);
await wallet.prettyPrintBalances(tokens);

wallet sync:2708a49abc4d6464044abfb4b9bce3af69a2b55d syncNonce 0, ethNonce 0
Balance of 0x36615Cf349d7F6344891B1e7CA7C72883F5dc049 ( sync:2708a49abc4d6464044abfb4b9bce3af69a2b55d ):
┌─────────┬───────┬───────┬──────────┐
│ (index) │  eth  │ sync  │ withdraw │
├─────────┼───────┼───────┼──────────┤
│   ETH   │ '0.0' │ '0.0' │  '0.0'   │
└─────────┴───────┴───────┴──────────┘


In [7]:
const numDeposits = 100;
const promises = [];

// set up multibars
const multibar = new cliProgress.MultiBar({
    clearOnComplete: false,
    hideCursor: true
}, cliProgress.Presets.shades_grey);
const sentProgressBar    = multibar.create(numDeposits, 0);
const receiptProgressBar = multibar.create(numDeposits, 0);

// send deposits
for (const i of utils.range(numDeposits)) {
    if (i % 50 === 0) {
        if (await WalletDecorator.isExodus()) {
            break;
        }
    }

    const receiptUpdater = i => res => {
        receiptProgressBar.update(i);
        return res;
    };

    const amount = Math.random() < 0.5 ? DEPOSIT_AMOUNT.div(numDeposits) : ethers.utils.parseEther('1000');
    const promise = wallet
        .deposit(amount, tokens)
        .then(receiptUpdater(i + 1));

    promises.push(promise);
    sentProgressBar.update(i + 1);
}

// get results
const results = utils.flat(await Promise.all(promises));
multibar.stop();

 ████████████████████░░░░░░░░░░░░░░░░░░░░ 50% | ETA: 0s | 50/100
 ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 0% | ETA: 0s | 0/100 ████████████████████████████████████████ 100% | ETA: 0s | 100/100



 ██████████████░░░░░░░░░░░░░░░░░░░░░░░░░░ 36% | ETA: 1s | 36/100
 ████████████████████░░░░░░░░░░░░░░░░░░░░ 50% | ETA: 1s | 50/100
 ██████████████████████████████████████░░ 94% | ETA: 1s | 94/100


In [16]:
console.log('results:', results.length/* , results.slice(-1) */);
const sentResults = results.filter(tx => !tx.error);
console.log('sentResults:', sentResults.length/* , sentResults.slice(-1) */);
const sentResultsPayloadless = sentResults.filter(tx => !(tx && tx.payload && tx.payload.token));
const sentResultsOk = sentResults.filter(tx => !tx.error);
console.log('sentResultsPayloadless:', sentResultsPayloadless.length, sentResultsPayloadless.slice(-1));

results: 100
sentResults: 0
sentResultsPayloadless: 0 []


In [20]:
sentResultsOk.map(({ payload: { amount } }) => ethers.utils.formatEther(amount))

[]


In [ ]:
// compute total deposit amount for each token
const sentAmounts = {};
for (const { payload: { token, amount } } of sentResultsOk) {
    sentAmounts[tokezn] = 
        (sentAmounts[token] || ethers.utils.bigNumberify(0)).add(amount);
}
for (const token of Object.keys(sentAmounts)) {
    sentAmounts[token] = ethers.utils.formatEther(sentAmounts[token]);
}
console.log({sentAmounts});

// cancel all deposits
await WalletDecorator.waitExodus();
await wallet.callCancelOutstandingDepositsForExodusModeNTimes(20);

// check that all sent deposits can be withdrawn
const withdrawBalances = await wallet.balancesToWithdraw(tokens);
for (const token of Object.keys(sentAmounts)) {
    const ok = sentAmounts[token] === withdrawBalances[token];
    console.log(`For ${token}:`, ok ? 'All good 👌' : `${sentAmounts[token]} ≠ ${withdrawBalances[token]}`);
    // assert(ok);
}

await wallet.prettyPrintBalances(tokens);
console.log('🎓 the end. Press ctrl c');
}